In [1]:
!pip install openai --quiet

In [5]:
import openai

# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

In [6]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


## Talk directly to ChatGPT and try and get back a response

In [7]:
openai.api_key = os.environ["OPENAI_API_KEY"]

response = openai.ChatCompletion.create(
  model=GPT_MODEL,
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the Gold medal in men's curling in the Winter Olympics 2022?"},
    ]
)

print(response['choices'][0]['message']['content'])

I'm sorry, but I cannot provide real-time information as I don't have access to the internet. However, you can easily find the answer by searching online or checking the official Winter Olympics website.


## Get the data about the Winter Olympics and provide the information to ChatGPT as context

In [8]:
!pip install matplotlib --quiet
!pip install plotly.express --quiet
!pip install scikit-learn --quiet
!pip install tabulate --quiet
!pip install tiktoken --quiet
!pip install wget --quiet

In [9]:
import pandas as pd
import wget
import ast

## 1. Grab the data from CSV and prepare it

In [10]:
# download pre-chunked text and pre-computed embeddings
# this file is ~200 MB, so may take a minute depending on your connection speed
embeddings_path = "https://cdn.openai.com/API/examples/data/winter_olympics_2022.csv"
file_path = "winter_olympics_2022.csv"

if not os.path.exists(file_path):
    wget.download(embeddings_path, file_path)
    print("File downloaded successfully.")
else:
    print("File already exists in the local file system.")

File downloaded successfully.


In [11]:
df = pd.read_csv(
    "winter_olympics_2022.csv"
)

# convert embeddings from CSV str type back to list type
df['embedding'] = df['embedding'].apply(ast.literal_eval)

In [13]:
df

,text,embedding
0,Lviv bid for the 2022 Winter Olympics\n\n{{Oly...,"[-0.005021067801862955, 0.00026050032465718687..."
1,Lviv bid for the 2022 Winter Olympics\n\n==His...,"[0.0033927420154213905, -0.007447326090186834,..."
2,Lviv bid for the 2022 Winter Olympics\n\n==Ven...,"[-0.00915789045393467, -0.008366798982024193, ..."
3,Lviv bid for the 2022 Winter Olympics\n\n==Ven...,"[0.0030951891094446182, -0.006064314860850573,..."
4,Lviv bid for the 2022 Winter Olympics\n\n==Ven...,"[-0.002936174161732197, -0.006185177247971296,..."
...,...,...
6054,Anaïs Chevalier-Bouchet\n\n==Personal life==\n...,"[-0.027750400826334953, 0.001746018067933619, ..."
6055,Uliana Nigmatullina\n\n{{short description|Rus...,"[-0.021714167669415474, 0.016001321375370026, ..."
6056,Uliana Nigmatullina\n\n==Biathlon results==\n\...,"[-0.029143543913960457, 0.014654331840574741, ..."
6057,Uliana Nigmatullina\n\n==Biathlon results==\n\...,"[-0.024266039952635765, 0.011665306985378265, ..."


In [14]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6059 entries, 0 to 6058
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       6059 non-null   object
 1   embedding  6059 non-null   object
dtypes: object(2)
memory usage: 94.8+ KB


## 2. Set up SingleStoreDB

In [15]:
%%sql

-- Create the database
DROP DATABASE IF EXISTS winter_wikipedia;
CREATE DATABASE IF NOT EXISTS winter_wikipedia;

In [16]:
%%sql

USE winter_wikipedia;
CREATE TABLE IF NOT EXISTS winter_olympics_2022 (
    id INT PRIMARY KEY,
    text TEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
    embedding BLOB
);

## 3. Populate the Table with our dataframe and use JSON_ARRAY_PACK

In [17]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [18]:
def insert_records(
    df: pd.DataFrame,
    table_name: str,
    batch_size: int = 1000
):

    stmt = f"""
        INSERT INTO {table_name} (
            id,
            text,
            embedding
        )
        VALUES (
            %s,
            %s,
            JSON_ARRAY_PACK(%s)
        )
    """.format(table_name=table_name)
    
    record_arr = df.to_records(index=True)
    
    for i in range(0, len(record_arr), batch_size):
        batch = record_arr[i:i+batch_size]
        values = [(row[0], row[1], str(row[2])) for row in batch]
        db_connection.execute(stmt, values)
    return

insert_records(df, "winter_olympics_2022")

## 4. Do a semantic search with the same question from above and use the response to send to ChatGPT again

In [41]:
from openai.embeddings_utils import get_embedding

def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    table_name: str,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple:
    """Returns a list of strings and relatednesses, sorted from most related to least."""

    # Get the embedding of the query.
    query_embedding_response = get_embedding(query, EMBEDDING_MODEL)

    # Create the SQL statement.
    stmt = f"""
        SELECT
            text,
            DOT_PRODUCT(JSON_ARRAY_PACK(%s), embedding) AS score
        FROM {table_name}
        ORDER BY score DESC
        LIMIT %s
    """.format(table_name=table_name)

    # Execute the SQL statement.
    results = db_connection.execute(stmt, [str(query_embedding_response), top_n])

    strings = []
    relatednesses = []

    for row in results:
        strings.append(row[0])
        relatednesses.append(row[1])

    # Return the results.
    return strings[:top_n], relatednesses[:top_n]

In [42]:
from tabulate import tabulate

strings, relatednesses = strings_ranked_by_relatedness(
    "curling gold medal",
    df,
    "winter_olympics_2022",
    top_n=5
)

for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    print(tabulate([[string]], headers=['Result'], tablefmt='fancy_grid'))

relatedness=0.879
╒═══════════════════════════════════════════════════╕
│ Result                                            │
╞═══════════════════════════════════════════════════╡
│ Curling at the 2022 Winter Olympics               │
│                                                   │
│ ==Medal summary==                                 │
│                                                   │
│ ===Medal table===                                 │
│                                                   │
│ {{Medals table                                    │
│  | caption        =                               │
│  | host           =                               │
│  | flag_template  = flagIOC                       │
│  | event          = 2022 Winter                   │
│  | team           =                               │
│  | gold_CAN = 0 | silver_CAN = 0 | bronze_CAN = 1 │
│  | gold_ITA = 1 | silver_ITA = 0 | bronze_ITA = 0 │
│  | gold_NOR = 0 | silver_NOR = 1 | bronze_NOR = 0 │
│  | gold_

In [43]:
import tiktoken

def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from SingleStoreDB."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df, "winter_olympics_2022")
    introduction = 'Use the below articles on the 2022 Winter Olympics to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a table of relevant texts and embeddings in SingleStoreDB."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions about the 2022 Winter Olympics."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

In [44]:
from pprint import pprint

answer = ask("Who won the Gold medal in men's curling in the Winter Olympics 2022?")

pprint(answer)

("The team from Sweden won the Gold medal in men's curling in the Winter "
 'Olympics 2022.')
